In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import csv
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier 
import nltk
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud


In [3]:
train_df = pd.read_csv("train.csv")
train_df.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [5]:
X = train_df.drop(["id"],axis=1)
X.head()


,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
key = X["keyword"].value_counts().index[0]
print(key)
loc = X["location"].value_counts().index[0]
print(loc)


fatalities
USA


In [7]:
train_df['keyword'] = train_df['keyword'].fillna(train_df['keyword'].value_counts().idxmax())
train_df['location'] = train_df['location'].fillna(train_df['location'].value_counts().idxmax())
train_df.isnull().sum()


id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [8]:
X_new = train_df.drop(["target"],axis=1)
X_new.head()


,id,keyword,location,text
0,1,fatalities,USA,Our Deeds are the Reason of this #earthquake M...
1,4,fatalities,USA,Forest fire near La Ronge Sask. Canada
2,5,fatalities,USA,All residents asked to 'shelter in place' are ...
3,6,fatalities,USA,"13,000 people receive #wildfires evacuation or..."
4,7,fatalities,USA,Just got sent this photo from Ruby #Alaska as ...


In [9]:
y = train_df["target"]
y.shape


(7613,)

In [10]:
test_df.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [12]:
test_df['keyword'] = test_df['keyword'].fillna(test_df['keyword'].value_counts().idxmax())
test_df['location'] = test_df['location'].fillna(test_df['location'].value_counts().idxmax())
test_df.isnull().sum()

id          0
keyword     0
location    0
text        0
dtype: int64

In [13]:
X_new = X_new.drop(["id"],axis=1)
X_new.head()

,keyword,location,text
0,fatalities,USA,Our Deeds are the Reason of this #earthquake M...
1,fatalities,USA,Forest fire near La Ronge Sask. Canada
2,fatalities,USA,All residents asked to 'shelter in place' are ...
3,fatalities,USA,"13,000 people receive #wildfires evacuation or..."
4,fatalities,USA,Just got sent this photo from Ruby #Alaska as ...


In [14]:
X_new.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
X_new.head()


,keyword,location,text
0,fatalities,USA,Our Deeds are the Reason of this earthquake M...
1,fatalities,USA,Forest fire near La Ronge Sask Canada
2,fatalities,USA,All residents asked to shelter in place are ...
3,fatalities,USA,people receive wildfires evacuation or...
4,fatalities,USA,Just got sent this photo from Ruby Alaska as ...


In [15]:
for i in X_new.columns:
    X_new[i] = X_new[i].str.lower()
X_new.head()


,keyword,location,text
0,fatalities,usa,our deeds are the reason of this earthquake m...
1,fatalities,usa,forest fire near la ronge sask canada
2,fatalities,usa,all residents asked to shelter in place are ...
3,fatalities,usa,people receive wildfires evacuation or...
4,fatalities,usa,just got sent this photo from ruby alaska as ...


In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
X_new['keyword'].apply(lambda x: [item for item in x if item not in stop])
X_new['location'].apply(lambda x: [item for item in x if item not in stop])
X_new['text'].apply(lambda x: [item for item in x if item not in stop])
print(X_new.shape)


(7613, 3)


In [18]:
ori_test = test_df.drop(["id"],axis=1)

ori_test.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
ori_test.head()


,keyword,location,text
0,deluged,New York,Just happened a terrible car crash
1,deluged,New York,Heard about earthquake is different cities s...
2,deluged,New York,there is a forest fire at spot pond geese are...
3,deluged,New York,Apocalypse lighting Spokane wildfires
4,deluged,New York,Typhoon Soudelor kills in China and Taiwan


In [20]:
for i in ori_test.columns:
    ori_test[i] = ori_test[i].str.lower()
ori_test.head()

,keyword,location,text
0,deluged,new york,just happened a terrible car crash
1,deluged,new york,heard about earthquake is different cities s...
2,deluged,new york,there is a forest fire at spot pond geese are...
3,deluged,new york,apocalypse lighting spokane wildfires
4,deluged,new york,typhoon soudelor kills in china and taiwan


In [21]:
ori_test['keyword'].apply(lambda x: [item for item in x if item not in stop])
ori_test['location'].apply(lambda x: [item for item in x if item not in stop])
ori_test['text'].apply(lambda x: [item for item in x if item not in stop])
print(ori_test.shape)

(3263, 3)


In [22]:
X_new["sentence"] = X_new['keyword'] + " " + X_new['text']
train_text = np.array(X_new["sentence"])
print(train_text[0])

fatalities our deeds are the reason of this  earthquake may allah forgive us all


In [23]:
# test dataset
ori_test["sentence"] = ori_test['keyword'] + " " + ori_test['text']
test_text = np.array(ori_test["sentence"])
print(test_text[0])

deluged just happened a terrible car crash


In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_text,y,test_size=0.25,random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5709,)
(1904,)
(5709,)
(1904,)


In [25]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x_train)

<5709x18272 sparse matrix of type '<class 'numpy.float64'>'
	with 85020 stored elements in Compressed Sparse Row format>

In [26]:
keyword = vectorizer.get_feature_names()
X_train = vectorizer.transform(x_train)
X_test = vectorizer.transform(x_test)
X_test_new = vectorizer.transform(test_text)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
print(x_train[0])
print("\n")
print(X_train[0])


oil   spill refugio oil spill may have been costlier bigger than projected  a plains all american pipeline oil spill off     http   t co yhmregauxz


  (0, 17861)	0.2565668314084014
  (0, 15442)	0.14366087248498907
  (0, 14571)	0.5237386817894252
  (0, 12885)	0.19060966227864334
  (0, 12192)	0.1892120760961464
  (0, 11828)	0.22068164338909357
  (0, 11771)	0.22509726411545425
  (0, 11041)	0.5122637278245114
  (0, 10986)	0.14658635497872421
  (0, 9579)	0.15431815773620025
  (0, 7032)	0.04966717003573883
  (0, 6568)	0.11481042289808198
  (0, 3018)	0.19060966227864334
  (0, 2699)	0.04697600409067743
  (0, 1419)	0.1892120760961464
  (0, 1261)	0.1363386758557607
  (0, 472)	0.18787951794292748
  (0, 392)	0.1270409455452105


In [28]:
rc = RandomForestClassifier(max_depth = 400 ,random_state=0, n_estimators = 300)
rc.fit(X_train,y_train) 

RandomForestClassifier(max_depth=400, n_estimators=300, random_state=0)

In [29]:
pred1 = rc.predict(X_test)
matrix = confusion_matrix(y_test,pred1)
print(matrix)
score = accuracy_score(y_test,pred1)
print(score)
report = classification_report(y_test,pred1)
print(report)
prediction1 = rc.predict(X_test_new)
prediction1

[[1018   89]
 [ 321  476]]
0.7846638655462185
              precision    recall  f1-score   support

           0       0.76      0.92      0.83      1107
           1       0.84      0.60      0.70       797

    accuracy                           0.78      1904
   macro avg       0.80      0.76      0.77      1904
weighted avg       0.79      0.78      0.78      1904



array([0, 0, 1, ..., 1, 0, 0])

In [30]:
data = {'id':test_df["id"],'target':prediction1}
output = pd.DataFrame(data, columns = ['id','target'])
output.index = test_df.index

In [31]:
output.to_csv("submission.csv", index = False)    


In [32]:
a = pd.read_csv("submission.csv")
a

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,0
